In [1]:
import h5py
import numpy as np

f = h5py.File("chr1_pbmc_nbr_0_impute.hdf5", 'r')
coords = f["coordinates"]
xs, ys = coords[:,0].astype(int), coords[:,1].astype(int)

matrix_size = max(xs.max(), ys.max()) + 1

cell_keys = [k for k in f.keys() if k != "coordinates"]
n_vectors = len(cell_keys)

# Precompute indices for upper triangle including diagonal
triu_idx = np.triu_indices(matrix_size, k=0)
n_elements = len(triu_idx[0])

# Initialize array for flattened upper triangles
arr = np.zeros((n_vectors, n_elements))

# Fill each cell
for i, key in enumerate(cell_keys):
    data = f[key][:]  # values corresponding to coords
    # Construct full matrix
    mat = np.zeros((matrix_size, matrix_size))
    mat[xs, ys] = data
    mat[ys, xs] = data  # make symmetric
    arr[i] = mat[triu_idx]  # flatten upper triangle

print("arr shape:", arr.shape)  # (n_vectors, n_elements)


arr shape: (8681, 31125)


In [ ]:
import WeightedSigmoid as ws

weight_matrix = ws.construct_weights(matrix_size, start_weight=1.0, step_size=0.5)
cosine_correlation = ws.weighted_cosine(arr, weight_matrix)
pearson_correlation = ws.pearson_correlation(arr, weight_matrix)

In [ ]:
print(cosine_correlation)

In [ ]:
print(pearson_correlation)